In [1]:
#Include all the libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from statistics import mean
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import string 

#Sklearn Libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer

#Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import gensim.downloader as api

# Load the pre-trained Word2Vec model
model = api.load("glove-twitter-25")

#train test split and MLP model call
from sklearn.model_selection import train_test_split
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rudraksh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read training dataset

In [3]:
train_txt = pd.read_csv("train.csv").fillna('unknown')
train_txt = train_txt.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1)
train_txt.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


## Clean function

In [23]:
def clean_text(text):
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove numbers
        text = ''.join(word for word in text if not word.isdigit())

        # Remove double quotes
        text = text.replace('"', '')
        text = text.replace('“', '')
        text = text.replace('”', '')

        # Remove single quotes
        text = text.replace("'", "")
        text = text.replace("’","")

        #Removing single character
        """text = [word for word in text if len(word) > 1]
        text = ' '.join(text)"""

        # Remove special characters
        text = text.replace('\r', '').replace('\n', ' ').replace('\t', '').strip()
        
        # Tokenize the text
        tokens = word_tokenize(text)
        
        # Remove stop words
        tokens = [word for word in tokens if not word.lower() in stopwords.words('english')]
        
        # Join the tokens back into a string
        clean_text = ' '.join(tokens)
        return clean_text

In [ ]:
clean_text(train_data['comment_tsxt'])
clean_text()

In [7]:
train_txt['comment_text'] = train_txt['comment_text'].apply(clean_text)

In [8]:
train_txt.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation edits made username Hardcore Metal...,0
1,000103f0d9cfb60f,Daww matches background colour Im seemingly st...,0
2,000113f07ec002fd,Hey man Im really trying edit war guy constant...,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0
4,0001d958c54c6e35,sir hero chance remember page thats,0


In [9]:
train_txt.to_csv('clean_train.csv', index=False)

In [2]:
train_txt_clean = pd.read_csv("clean_train.csv")

In [3]:
train_txt_clean.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation edits made username Hardcore Metal...,0
1,000103f0d9cfb60f,Daww matches background colour Im seemingly st...,0
2,000113f07ec002fd,Hey man Im really trying edit war guy constant...,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0
4,0001d958c54c6e35,sir hero chance remember page thats,0


# Creating the embedding columns

In [4]:
import numpy as np
import pandas as pd
import gensim

def create_embeddings(df,column):
    # Load the pre-trained Word2Vec model
    # Create embeddings for each text column in the DataFrame
    if column in df.columns:
        embeddings = []
        for text in df[column]:
            if isinstance(text, str):
                words = text.split()
                embedding = np.zeros(model.vector_size)
                count = 0
                for word in words:
                    if word in model:
                        embedding += model[word]
                        count += 1
                if count != 0:
                    embedding /= count
                embeddings.append(embedding)
            else:
                embeddings.append(np.zeros(model.vector_size))
        df_copy = df.copy()
        df_copy['comment_text_embedding'] = embeddings
        return df_copy
    
    return df



In [5]:
train_txt_embedding = create_embeddings(train_txt_clean,'comment_text')

In [6]:
train_txt_embedding.head()

,id,comment_text,toxic,comment_text_embedding
0,0000997932d777bf,Explanation edits made username Hardcore Metal...,0,"[0.13968400185306867, 0.4844912625849247, 0.22..."
1,000103f0d9cfb60f,Daww matches background colour Im seemingly st...,0,"[-0.004671670496463776, -0.19248167425394058, ..."
2,000113f07ec002fd,Hey man Im really trying edit war guy constant...,0,"[0.17575090415775776, 0.35429790448397397, -0...."
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0,"[0.24397815026818437, 0.2689453430146851, -0.0..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,"[0.15511666610836983, 0.21799283660948277, 0.0..."


In [7]:
train_txt_embedding["comment_text_embedding"].apply(len).min()

25

In [19]:


def train_MLP_model(train_data, train_labels, num_layers=2):
    
    mlp_model = MLPClassifier(hidden_layer_sizes=[100]*(num_layers), max_iter=10, random_state=42)
    mlp_model.fit(train_data, train_labels)
    return mlp_model

# train the  model 

In [20]:
from keras_preprocessing.sequence import pad_sequences
X = train_txt_embedding.comment_text_embedding.tolist()
y =train_txt_embedding.toxic.tolist()

# Split the data into training and validation sets
#train_data, val_data, train_labels, val_labels = train_test_split(X,y,test_size =0.2,random_state=42)

# Pad the sequences with zeros to make them all the same length
max_seq_len = max(len(seq) for seq in X)
padded_train_data = pad_sequences(X, maxlen=max_seq_len, dtype='float32', padding='post', truncating='post', value=0.0)
#padded_val_data = pad_sequences(val_data, maxlen=max_seq_len, dtype='float32', padding='post', truncating='post', value=0.0)


#print(train_data)
#print(padded_train_data)
mlp_model = train_MLP_model(padded_train_data, y, num_layers=2)
print("model trained")
# Evaluate the model on the validation set
#score = mlp_model.score(padded_val_data, val_labels)
#print('MLP model accuracy:', score)

model trained


/Users/rudraksh/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


# Testing the data

In [59]:
test_txt = pd.read_csv("test.csv").fillna('unknown')



In [60]:
test_txt['comment_text']= test_txt['comment_text'].apply(clean_text)

In [ ]:
test_txt.to_csv('clean_test.csv', index=False)

In [58]:
test_txt.head()

,id,comment_text,toxic_prob,class prediction
0,00001cee341fdb12,Yo bitch Ja Rule succesful youll ever whats ha...,0.945796,toxic
1,0000247867823ef7,RfC title fine IMO,0.049498,non-toxic
2,00013b17ad220c46,Sources Zawe Ashton Lapland —,0.021680,non-toxic
3,00017563c3f7919a,look back source information updated correct f...,0.001010,non-toxic
4,00017695ad8997eb,dont anonymously edit articles,0.059091,non-toxic


In [52]:
test_label = pd.read_csv("test_labels.csv")

In [53]:
#embedding for test_data
test_txt_embedding = create_embeddings(test_txt,'comment_text')


In [54]:
test_txt_embedding.head()

,id,comment_text,comment_text_embedding
0,00001cee341fdb12,Yo bitch Ja Rule succesful youll ever whats ha...,"[0.025626862182674046, 0.4838857632252927, 0.2..."
1,0000247867823ef7,RfC title fine IMO,"[-0.030429989099502563, 0.3082050010561943, -0..."
2,00013b17ad220c46,Sources Zawe Ashton Lapland —,"[0.5126399993896484, -1.1583000421524048, -0.2..."
3,00017563c3f7919a,look back source information updated correct f...,"[0.21912502170016523, 0.38904624804854393, -0...."
4,00017695ad8997eb,dont anonymously edit articles,"[0.6702792663127184, 0.4176885075867176, -0.31..."


In [55]:
test_label.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [56]:
#predict model for test 
# converting to list
test_val = test_txt_embedding.comment_text_embedding.tolist()
test_label =test_label.toxic.tolist()

#calculate the accuracy 
max_seq_len = max(len(seq) for seq in X)
padded_test_data = pad_sequences(test_val, maxlen=max_seq_len, dtype='float32', padding='post', truncating='post', value=0.0)
# Evaluate the model on the validation set

#toxic prob
prob = mlp_model.predict_proba(padded_test_data)
toxic_prob = prob[:,1]
test_txt['toxic_prob'] = toxic_prob

prediction = mlp_model.predict(padded_test_data)
prediction = np.round(prediction).astype(int)
prediction = np.where(prediction == 0, 'non-toxic', 'toxic')
test_txt['class prediction'] = prediction

score = mlp_model.score(padded_test_data, test_label)
print('MLP model accuracy:', score)

MLP model accuracy: 0.3818260165574156


In [57]:
test_txt.head()

,id,comment_text,toxic_prob,class prediction
0,00001cee341fdb12,Yo bitch Ja Rule succesful youll ever whats ha...,0.945796,toxic
1,0000247867823ef7,RfC title fine IMO,0.049498,non-toxic
2,00013b17ad220c46,Sources Zawe Ashton Lapland —,0.021680,non-toxic
3,00017563c3f7919a,look back source information updated correct f...,0.001010,non-toxic
4,00017695ad8997eb,dont anonymously edit articles,0.059091,non-toxic
